In [2]:
!pip install optuna
!pip install gpytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [5]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from functools import reduce
import os
import joblib
import pickle
from pathlib import Path

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import gpytorch
from gpytorch.kernels import MaternKernel, PeriodicKernel, LinearKernel, ScaleKernel, MultitaskKernel, RBFKernel, AdditiveKernel
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution, VariationalStrategy, MultitaskVariationalStrategy, LMCVariationalStrategy
from gpytorch.distributions import MultitaskMultivariateNormal
from gpytorch.likelihoods import MultitaskGaussianLikelihood
from gpytorch.mlls import VariationalELBO
from gpytorch.means import ConstantMean

from joblib import Parallel, delayed
import gc
from scipy.stats import multivariate_normal
import optuna
from optuna.samplers import TPESampler
from typing import Tuple
from functools import partial
from linear_operator.utils.errors import NotPSDError

In [6]:
#Laden der Rowwise Daten da bessser geeignet
rowwiseDf = pd.read_csv("/kaggle/input/rowwisedfcsv/rowwiseDf.csv")

In [8]:
gpytorch.settings.cholesky_jitter._global_default = 1e-2

In [9]:
#Input Parameters
randomState= 21
nJobs= -1
nTrials= 20
#topK= 3
notebook_dir = Path().resolve()
storagePath = notebook_dir.parent / "data" / "rowWiseGPModel"
Stations= ["Erfurt-Weimar", "Schmücke", "Eisenach", "Artern", "Neuhaus am Rennweg","Meiningen", "Leinefelde", "Osterfeld"]
ReducedJahre= 5
minInducing= 1000
SAVE_DIR = "/kaggle/working/gp_checkpoints"

selected_Station = ["Osterfeld"] #LastYear wenn letztes Jahr
features = ['Stationshoehe','geoBreite','geoLaenge', 'hour', 'day_of_year', 'time_hours']
targets = ['TT_TU','RF_TU','  R1','  P0','   F']

In [10]:
rowwiseDf['MESS_DATUM']= pd.to_datetime(rowwiseDf['MESS_DATUM'])

rowwiseDf['day_of_year']= rowwiseDf['MESS_DATUM'].dt.dayofyear

In [11]:
rowwiseDf.columns

Index(['MESS_DATUM', 'STATIONS_ID', 'TT_TU', 'RF_TU', '  R1', 'RS_IND', 'WRTR',
       '   P', '  P0', '   F', '   D', 'Stationshoehe', 'geoBreite',
       'geoLaenge', 'Stationsname', 'hour', 'day', 'month', 'hour_sin',
       'hour_cos', 'month_sin', 'month_cos', 'day_of_year_sin',
       'day_of_year_cos', 'day_of_year'],
      dtype='object')

In [16]:
distanceDf= rowwiseDf[['geoBreite', 'geoLaenge', 'Stationshoehe', 'Stationsname', 'STATIONS_ID']].drop_duplicates()

In [17]:
print(distanceDf)

         geoBreite  geoLaenge  Stationshoehe          Stationsname  \
0          50.3066    10.9679            344  Lautertal-Oberlauter   
87648      51.7234    10.6021            607             Braunlage   
175296     51.5002     9.9507            167             Göttingen   
262944     50.3123    11.8760            565                   Hof   
350592     50.4973     9.9427            920           Wasserkuppe   
438240     50.9829    10.9608            316         Erfurt-Weimar   
525888     50.8520     9.7377            272         Hersfeld, Bad   
613536     50.4818    12.1300            387                Plauen   
701184     51.8454    10.7686            233           Wernigerode   
788832     51.3932    10.3123            356            Leinefelde   
876480     51.3744    11.2920            164                Artern   
964128     50.8812    12.1289            311         Gera-Leumnitz   
1051776    50.6545    10.7696            938              Schmücke   
1139424    50.5611  

In [13]:
lat1 = None
lon1 = None
for _, row in distanceDf.iterrows():
    if row['Stationsname'] == 'Erfurt-Weimar':
        lat1 = np.radians(row['geoBreite'])
        lon1 = np.radians(row['geoLaenge'])
        break  # reicht, wenn wir den ersten passenden Eintrag haben

# 2. Distanz für alle anderen Stationen berechnen
distances = []
for _, row in distanceDf.iterrows():
    if row['Stationsname'] == 'Erfurt-Weimar':
        continue

    lat2 = np.radians(row['geoBreite'])
    lon2 = np.radians(row['geoLaenge'])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    distance = 2 * 6371 * np.arcsin(np.sqrt(a))  # in km

    distances.append({
        'Stationsname': row['Stationsname'],
        'distance_km': distance
    })

# 3. In DataFrame umwandeln und sortieren
distance_result = pd.DataFrame(distances).sort_values('distance_km')
print(distance_result)

            Stationsname  distance_km
11              Schmücke    38.908361
17              Eisenach    41.949378
9                 Artern    49.275834
15    Neuhaus am Rennweg    55.036985
12             Meiningen    62.326012
8             Leinefelde    64.219198
14             Osterfeld    68.707104
16               Schleiz    75.136254
0   Lautertal-Oberlauter    75.202795
13            Harzgerode    75.396944
10         Gera-Leumnitz    82.636570
1              Braunlage    86.024839
5          Hersfeld, Bad    86.966964
4            Wasserkuppe    89.710348
2              Göttingen    90.844191
7            Wernigerode    96.827360
3                    Hof    98.609755
6                 Plauen    99.376594


In [ ]:
#reduziere Daten größe damit Modell berechnbar bleibt
reducedDf= rowwiseDf.copy()
reducedDf['MESS_DATUM']= pd.to_datetime(reducedDf['MESS_DATUM'])

maxDate= reducedDf['MESS_DATUM'].max()
cutoffDate= maxDate - pd.DateOffset(years= ReducedJahre)

reducedDf= reducedDf[(reducedDf['Stationsname'].isin(Stations)) & (reducedDf['MESS_DATUM'] >= cutoffDate)]

In [ ]:
reducedDf['time_hours'] = (reducedDf['MESS_DATUM'] - reducedDf['MESS_DATUM'].min()).dt.total_seconds() / 3600

In [ ]:
#month wird gedropt da hoch koriliert mit day of year

In [ ]:
reducedDf= reducedDf.drop(columns= ['STATIONS_ID', 'RS_IND', 'WRTR','   P', '   D', 'day', 'month', 'month_sin', 'month_cos', 'hour_sin', 'hour_cos', 'day_of_year_sin', 'day_of_year_cos'])

In [ ]:
reducedDf.columns

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
os.makedirs(SAVE_DIR, exist_ok=True)

In [ ]:
for station in selected_Station:
    os.makedirs(SAVE_DIR + '/' + station, exist_ok=True)

In [ ]:
class MultitaskGPModel(ApproximateGP):
    def __init__(self, num_latents, num_tasks, inducing_points, nu_height, nu_lonlat, daily_period, yearly_period):
        variational_distribution= gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2), batch_shape=torch.Size([num_latents])
        )

        variational_strategy= gpytorch.variational.LMCVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ),
            num_tasks= num_tasks,
            num_latents= num_latents,
            latent_dim= -1
        )

        super().__init__(variational_strategy)

        self.mean_module= gpytorch.means.ConstantMean(batch_shape= torch.Size([num_latents]))

        self.daily_kernel= PeriodicKernel(period_length=daily_period, batch_shape= torch.Size([num_latents]), active_dim=[3])
        self.yearly_kernel= PeriodicKernel(period_length=yearly_period, batch_shape= torch.Size([num_latents]), active_dim=[4])
        self.hight_kernel= MaternKernel(nu=nu_height, batch_shape= torch.Size([num_latents]), active_dim=[0])
        self.lonLat_kernel= MaternKernel(nu=nu_lonlat, batch_shape= torch.Size([num_latents]), active_dim=[1,2])
        self.conter_kernel= RBFKernel(batch_shape= torch.Size([num_latents]), active_dim=[5])
        
        self.covar_module= ScaleKernel(
            AdditiveKernel(
                self.daily_kernel,
                self.yearly_kernel,
                self.lonLat_kernel,
                self.hight_kernel,
                self.conter_kernel
            ),
            batch_shape= torch.Size([num_latents])
        )

    def forward(self, x):
        mean_x= self.mean_module(x)
        covar_x= self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
def objective(trial, X_train_np, y_train_np, X_val_np, y_val_np,
              num_tasks=5, device=device, save_dir=SAVE_DIR, batch_size=512):
    # Optuna hyperparams (add or remove as you like)
    num_inducing = 1000
    nu_lonlat = 1.5
    nu_height = 0.5
    daily_period = 24
    yearly_period = 365.25
    lr = 0.0037427697892870536
    training_iter = 129
    num_latents = 6

    # Create inducing points (from X_sub)
    subset_idx = np.random.choice(X_train_np.shape[0], min(10000, X_train_np.shape[0]), replace=False)
    X_kmeans_np = X_train_np[subset_idx]

    kmeans = KMeans(n_clusters=num_inducing, n_init=10, random_state=randomState)
    kmeans.fit(X_kmeans_np)
    inducing_points = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32).to(device)

    # Convert to tensors
    X_sub = torch.tensor(X_train_np, dtype=torch.float32).to(device)
    y_sub = torch.tensor(y_train_np, dtype=torch.float32).to(device)

    dataset = TensorDataset(X_sub, y_sub)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Build model & likelihood
    # Pass period_lengths into kernel by setting them after init
    model = MultitaskGPModel(
        inducing_points=inducing_points,
        num_tasks=num_tasks,
        num_latents= num_latents,
        nu_lonlat=nu_lonlat,
        nu_height=nu_height,
        daily_period= daily_period,
        yearly_period= yearly_period
    ).to(device)

    likelihood = MultitaskGaussianLikelihood(num_tasks=num_tasks).to(device)

    # ⬇️ Multi-GPU Wrapper
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = torch.nn.DataParallel(model)
        likelihood = torch.nn.DataParallel(likelihood)
        
    # initialize noise to a sensible positive value to help stability
    try:
        likelihood.noise_covar.initialize(noise=0.05)
    except Exception:
        pass

    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam([{'params': model.parameters()}, {'params': likelihood.parameters()}], lr=lr)

    if isinstance(model, torch.nn.DataParallel):
        mll = VariationalELBO(likelihood.module, model.module, num_data=X_sub.shape[0])
    else:
        mll = VariationalELBO(likelihood, model, num_data=X_sub.shape[0])
    #mll = VariationalELBO(likelihood, model, num_data=X_sub.shape[0])

    best_loss = float("inf")
    accumulation_steps = 2 
    try:
        for it in range(training_iter):
            epoch_loss = 0.0
            scaler = GradScaler()
            for X_batch, y_batch in loader:
                optimizer.zero_grad()
                with autocast():
                    # increase jitter during training for robustness
                    with gpytorch.settings.cholesky_jitter(1e-3):
                        output = model(X_batch)
                        loss = -mll(output, y_batch)
                # catch NaN
                if not torch.isfinite(loss):
                    raise RuntimeError("Loss NaN or INF")

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                #loss.backward()
                #optimizer.step()
                epoch_loss += loss.item() * X_batch.size(0)

            # mittlere Epoche
            epoch_loss /= len(dataset)
            if epoch_loss < best_loss:
                best_loss = epoch_loss

            del X_batch, y_batch, output, loss
            torch.cuda.empty_cache()
            gc.collect()

        # Validation
        model.eval()
        likelihood.eval()
        total_val_loss = 0.0
        with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cholesky_jitter(1e-3):
            X_val = torch.tensor(X_val_np, dtype=torch.float32).to(device)
            y_val = torch.tensor(y_val_np, dtype=torch.float32).to(device)
            for start_idx in range(0, X_val.shape[0], batch_size):
                end_idx = min(start_idx + batch_size, X_val.shape[0])
                X_batch = torch.tensor(X_val_np[start_idx:end_idx], dtype=torch.float32).to(device)
                y_batch = torch.tensor(y_val_np[start_idx:end_idx], dtype=torch.float32).to(device)
                
                # Vorwärtsdurchlauf und Verlustberechnung
                val_output = model(X_batch)
                val_loss = -mll(val_output, y_batch).item()
        
                # Addiere den Verlust für diese Batch
                total_val_loss += val_loss * X_batch.size(0)
        
            # Berechne den durchschnittlichen Verlust über alle Batches
            average_val_loss = total_val_loss / X_val.shape[0]
    
    except NotPSDError as e:
        # numerical issue with PSD — return a large loss so Optuna avoids such hyperparams
        print("NotPSDError in trial:", e)
        return 1e6
    except Exception as e:
        print("Exception during training:", e)
        return 1e6

    # Save model state for this trial
    model_path = os.path.join(save_dir, f"model_trial_{trial.number}.pt")
    lik_path = os.path.join(save_dir, f"likelihood_trial_{trial.number}.pt")
    if isinstance(model, torch.nn.DataParallel):
        torch.save(model.module.state_dict(), model_path)
        # likelihood ist auch DataParallel → also auch .module
        torch.save(likelihood.module.state_dict(), lik_path)
    else:
        torch.save(model.state_dict(), model_path)
        torch.save(likelihood.state_dict(), lik_path)
    print(f"[Trial {trial.number}] saved to {model_path}")

    return average_val_loss

In [ ]:
for station in selected_Station:
    if station == 'LastYear':
        # Split nach Datum: letzte 10% als Test
        maxDate = reducedDf['MESS_DATUM'].max()
        cutoffDate = maxDate - pd.DateOffset(years=1)
        df_test = reducedDf[reducedDf['MESS_DATUM'] >= cutoffDate]
        df_train = reducedDf[reducedDf['MESS_DATUM'] < cutoffDate]
        station_name = "None"
    else:
        # Split nach Station
        df_test = reducedDf[reducedDf['Stationsname'] == station]
        df_train = reducedDf[reducedDf['Stationsname'] != station]
        station_name = station

    drop_cols = ['MESS_DATUM', 'Stationsname']
    df_train = df_train.drop(columns=drop_cols, errors='ignore')
    df_test = df_test.drop(columns=drop_cols, errors='ignore')

    X_train= df_train[features].values.astype(np.float32)
    y_train= df_train[targets].values.astype(np.float32)

    X_test= df_test[features].values.astype(np.float32)
    y_test= df_test[targets].values.astype(np.float32)

    scalerX = StandardScaler()
    X_train_np = scalerX.fit_transform(X_train)
    X_val_np = scalerX.transform(X_test)
    scalerY = StandardScaler()
    y_train_np = scalerY.fit_transform(y_train)
    y_val_np = scalerY.transform(y_test)

    objective_fn = partial(
        objective,
        X_train_np=X_train_np,
        y_train_np=y_train_np,
        X_val_np=X_val_np,
        y_val_np=y_val_np,
        num_tasks= len(targets),          # Anzahl Output-Tasks
        device=device,
        save_dir=SAVE_DIR + '/' + station
    )

    storage_path = f"sqlite:///{SAVE_DIR}/{station}.db"
    study = optuna.create_study(
        study_name="sparse_gp_weather",
        storage=storage_path,
        direction="minimize",
        load_if_exists=True,
    )
    study.optimize(objective_fn, n_trials=1)# Optuna-Studie mit persistentem Speicher